<a href="https://www.kaggle.com/code/seyed0123/chess?scriptVersionId=287622219" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# prossesing the data

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


def load_all_txt(input_dir):
    all_text = ""

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_dir, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                all_text += f.read() + "\n"

    return all_text

In [2]:
input_directory = r"/kaggle/input/chess-analyse-books"
full_text = load_all_txt(input_directory)
print(len(full_text))

5556076


# load and test model

In [3]:
!pip install transformers accelerate pyarrow==14.0.2 datasets==2.18.0 python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 103.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch

model_name = "Qwen/Qwen3-1.7B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float32,
    device_map="auto"
)
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [5]:
class StopAtSentence(StoppingCriteria):
    def __call__(self, input_ids, scores, **kwargs):
        # Check last token
        last_token = input_ids[0][-1].item()
        # Stop if EOS or common sentence-ending token (".", "?", "!")
        if last_token == tokenizer.eos_token_id:
            return True
        decoded = tokenizer.decode([last_token], skip_special_tokens=True)
        if decoded.strip() in {".", "?", "!"}:
            return True
        return False

def chat_qwen3_direct(prompt: str, max_new_tokens: int = 256) -> str:
    messages = [{"role": "user", "content": prompt}]
    
    # Apply chat template WITHOUT thinking
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # 🔥 Disable thinking for factual answers
    )
    
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,      # Lower = more focused
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # stopping_criteria=StoppingCriteriaList([StopAtSentence()])  # Optional
        )
    
    # Extract only the new text
    input_len = inputs.input_ids.shape[1]
    response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)
    
    # Clean: remove any trailing incomplete sentence
    for end in [".", "?", "!"]:
        if end in response:
            response = response[:response.rfind(end)+1]
            break
    else:
        # No sentence end? Return first line or truncate
        response = response.split("\n")[0].strip()
    
    return response.strip()

## Test the model output on simple question

In [6]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt)
print("💬 Answer:", answer)

💬 Answer: Controlling the center in chess is crucial because it gives a player a significant advantage in terms of piece mobility, development, and overall control of the board. Center control allows pieces to attack more effectively and develop more quickly, while also limiting the opponent's options. A strong center often leads to better coordination among pieces and increases the chances of winning.


# fine tune the model

In [7]:
def chunk_text(text, tokenizer, max_length):
    # Use batched tokenization to avoid huge tensors
    tokens = tokenizer(
        text,
        return_tensors="pt",
        add_special_tokens=False,  # Avoid extra BOS/EOS in middle of book
        truncation=False,
        verbose=False
    )["input_ids"][0]

    chunks = []
    # Use full chunks only (avoid tiny trailing fragments)
    for i in range(0, len(tokens) - max_length + 1, max_length):
        chunk = tokens[i:i + max_length].tolist()
        chunks.append({"input_ids": chunk})
    
    # Optional: add last partial chunk if long enough
    remainder = len(tokens) % max_length
    if remainder > 64:  # only if >64 tokens
        chunks.append({"input_ids": tokens[-remainder:].tolist()})
    
    return chunks

## prepare the data for training

In [8]:
from peft import LoraConfig, get_peft_model, TaskType,PeftModel
lora_config = LoraConfig(
    r=16,                # LoRA rank
    lora_alpha=32,       # LoRA alpha
    target_modules=["q_proj", "v_proj"],  # common modules for LLaMA/Qwen
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

max_length = 256

data = chunk_text(full_text, tokenizer, max_length)
dataset = Dataset.from_list(data)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,                              
    pad_to_multiple_of=8                    
)

In [10]:
training_args = TrainingArguments(
    output_dir="./chess-qwen",
    run_name="chess-qwen-run-1",

    
    per_device_train_batch_size=2,    
    gradient_accumulation_steps=16,   

   
    num_train_epochs=1,
    learning_rate=2e-4,                
    fp16=True,                      
    bf16=False,                        

    
    save_steps=2000,                   
    logging_steps=200,                  
    save_total_limit=1,                 

    # 🚀 Other speedups
    optim="adamw_torch",                
    report_to="none",                  
    dataloader_num_workers=4,           
    remove_unused_columns=True,
)


# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## training loop

In [11]:
print('start training')
trainer.train()

print("Merging LoRA adapters into base model...")
model = model.merge_and_unload()
model.save_pretrained(
    "./chess-qwen-adapted",
    safe_serialization=True
)
tokenizer.save_pretrained("./chess-qwen-adapted")

start training


Step,Training Loss
200,2.899300


Merging LoRA adapters into base model...


('./chess-qwen-adapted/tokenizer_config.json',
 './chess-qwen-adapted/special_tokens_map.json',
 './chess-qwen-adapted/chat_template.jinja',
 './chess-qwen-adapted/vocab.json',
 './chess-qwen-adapted/merges.txt',
 './chess-qwen-adapted/added_tokens.json',
 './chess-qwen-adapted/tokenizer.json')

In [12]:
del trainer
del model
del data_collator
del dataset
del tokenizer

torch.cuda.empty_cache()

# test the adapted model on same simple question

In [13]:
model_name = "/kaggle/working/chess-qwen-adapted"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [14]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt)
print("💬 Answer:", answer)

💬 Answer: Controlling the center in chess is crucial because it grants a player a significant advantage in terms of mobility, development, and the ability to exert influence over the game’s direction. It allows for greater flexibility in piece placement and often leads to stronger positional play. Additionally, the center is the most active and dynamic part of the board, making it an ideal location for developing pieces and exerting pressure on the opponent’s king.


# Quantizing the model using llama-cpp

In [15]:
!pip install sentencepiece
!pip install "llama-cpp-python>=0.2.0"
!git clone https://github.com/ggerganov/llama.cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl size=4503250 sha256=71d55eb363b1e60cd6c0c1af086e8fe6ccd066077768ea7f52186cd64bdbf45b
  Stored in directory: /root/.cache/pip/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating objects: 72940, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 72940 (delta 15), reused 7 (delta 7), pack-reused 72918 (from 2)
Receiving objects: 100% (72940/72940), 249.65 MiB | 19.15 MiB/s, done.
Resolving deltas: 100

In [16]:
!python llama.cpp/convert_hf_to_gguf.py \
    /kaggle/working/chess-qwen-adapted \
    --outfile /kaggle/working/chess-fp16.gguf \
    --outtype f16

Writing: 100%|███████████████████████████| 3.44G/3.44G [00:11<00:00, 309Mbyte/s]


In [17]:
!cd llama.cpp && mkdir -p build && cd build && cmake .. && make -j

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [18]:
!llama.cpp/build/bin/llama-quantize \
    /kaggle/working/chess-fp16.gguf \
    /kaggle/working/chess-q8_0.gguf \
    q8_0

main: build = 7499 (fd05c51ce)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing '/kaggle/working/chess-fp16.gguf' to '/kaggle/working/chess-q8_0.gguf' as Q8_0
llama_model_loader: loaded meta data with 29 key-value pairs and 310 tensors from /kaggle/working/chess-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_k i32              = 20
llama_model_loader: - kv   3:                     general.sampling.top_p f32              = 0.950000
llama_model_loader: - kv   4:                      general.sampling.temp f32              = 0.600000
llama_model_loader: - kv   5:                               general.name str              = Ches

# compare quantize model and two pipelines

## define the funcs

In [19]:
import chess
import chess.svg
import ipywidgets as widgets
from IPython.display import display

def board_widget(fen, size=350):
    board = chess.Board(fen)
    svg = chess.svg.board(board, size=size)
    return widgets.HTML(value=svg)


def labeled_board(title, fen):
    label = widgets.HTML(
        value=f"""
        <div style="
            text-align:center;
            font-weight:bold;
            margin-bottom:6px;
            font-size:16px;
        ">
            {title}
        </div>
        """
    )

    board = board_widget(fen)

    return widgets.VBox(
        [label, board],
        layout=widgets.Layout(align_items="center")
    )


def show_analysis(sample, llm_output):
    before_block = labeled_board("Before position", sample["before"]["fen"])
    after_block  = labeled_board("After position", sample["after"]["fen"])

    boards_row = widgets.HBox(
        [before_block, after_block],
        layout=widgets.Layout(
            justify_content="center",
            gap="40px"
        )
    )

    # Move type badge (color-coded)
    move_type = sample.get("move_type", "unknown").capitalize()
    move_colors = {
        "Blunder": "#d9534f",
        "Mistake": "#f0ad4e",
        "Inaccuracy": "#f7c948",
        "Normal": "#5bc0de",
        "Good": "#5cb85c"
    }
    color = move_colors.get(move_type, "#999")

    move_type_badge = widgets.HTML(
        value=f"""
        <div style="
            display:inline-block;
            margin:12px 0 6px 0;
            padding:4px 10px;
            border-radius:12px;
            font-size:13px;
            font-weight:bold;
            color:white;
            background:{color};
        ">
            {move_type}
        </div>
        """
    )

    # LLM text (left-aligned, symmetric)
    text = widgets.HTML(
        value=f"""
        <div style="
            margin-top:6px;
            padding:8px 12px;
            border:1px solid #ddd;
            border-radius:6px;
            font-family:Arial, sans-serif;
            font-size:14px;
            line-height:1.5;
            text-align:left;
            background:#fafafa;
        ">
            <div style="margin-left:16px;">
                {llm_output}
            </div>
        </div>
        """
    )

    display(
        widgets.VBox(
            [
                boards_row,
                widgets.HBox([move_type_badge], layout=widgets.Layout(justify_content="center")),
                text
            ]
        )
    )



In [20]:
position_features_prompt = """
You are a chess engine. 

Describe the position in terms of these aspects: 
1. Central control
2. Piece activity
3. King safety
4. Pawn structure
5. Initiative

Use 1 sentence per aspect.

Position:
{analysis}
"""

def llm_position_analysis(stockfish_text: str,llm_func) -> str:
    prompt = position_features_prompt.format(
        analysis=stockfish_text
    )
    return llm_func(prompt)

In [21]:
final_analyze_prompt = """
You are a chess position evaluator.

Win probabilities are from White’s perspective.

TASK:
Compare the position BEFORE and AFTER the move.
Decide whether the move improved or weakened the position,
and explain WHY.

RULES (must follow):
- Write only 2 or 3 sentences.
- Use plain natural language.
- Do NOT mention move names or chess notation.
- Do NOT mention numbers, evaluations, or engine terms.
- Write in a way that a chess beginner can understand.

Before position:
{before_position}

After position:
{after_position}

Position description BEFORE the move:
{position_description_before}

Position description AFTER the move:
{position_description_after}

Key position changes detected:

White:
{position_features_white}

Black:
{position_features_black}

Engine analysis context (for understanding only, do not mention directly):

Before move:
{before_stockfish}

After move:
{after_stockfish}

Checkmate status after the move:
{checkmate_status}

Now explain why this move made the position better or worse.
Focus on king safety, piece coordination, board control, and threats.
"""

def llm_move_analysis(before_llm, after_llm, sample, llm_func):
    prompt = final_analyze_prompt.format(
        before_position=before_llm,
        after_position=after_llm,
        position_description_before=sample.get("position_description_before", ""),
        position_description_after=sample.get("position_description_after", ""),
        position_features_white=sample.get("position_features_white", {}),
        position_features_black=sample.get("position_features_black", {}),
        before_stockfish=sample["before"]["stockfish_analysis"],
        after_stockfish=sample["after"]["stockfish_analysis"],
        checkmate_status=sample["checkmate"],
    )
    return llm_func(prompt)


In [22]:
def full_llm_pipeline(sample,llm_func, visualize=True):
    # Step 1: LLM evaluates positions independently
    before_llm = llm_position_analysis(sample["before"],llm_func)
    after_llm  = llm_position_analysis(sample["after"],llm_func)

    # Step 2: LLM compares its OWN outputs + Stockfish eval
    final_eval = llm_move_analysis(
        before_llm,
        after_llm,
        sample,
        llm_func
    )
    if visualize:
        show_analysis(sample, final_eval)

    return {
        "before_llm": before_llm,
        "after_llm": after_llm,
        "final_analysis": final_eval
    }
    
def simple_method(sample, llm_func, visualize=True):
    prompt = f"""
You are a chess coach explaining a move to a beginner.

IMPORTANT RULES (must follow):
- Write only 2 or 3 short sentences.
- Use simple, natural language.
- Do NOT mention move names or chess notation.
- Do NOT mention numbers, evaluations, or engine terms.
- Do NOT quote engine lines.
- Explain ideas, not calculations.

Position description BEFORE the move:
{sample.get("position_description_before", "")}

Position description AFTER the move:
{sample.get("position_description_after", "")}

Key position changes detected (this information is correct and important):

White:
{sample.get("position_features_white", {})}

Black:
{sample.get("position_features_black", {})}

Engine analysis context (for understanding only, do not mention directly):

Before move:
{sample["before"]["stockfish_analysis"]}

After move:
{sample["after"]["stockfish_analysis"]}

Checkmate status after the move:
{sample.get("checkmate", {})}

Now explain why this move weakened or improved the position.
Focus on king safety, piece coordination, control of the board, and threats.
"""
    final_eval = llm_func(prompt)

    if visualize:
        show_analysis(sample, final_eval)

    return final_eval



In [23]:
from llama_cpp import Llama

gguf_llm = Llama(
    model_path="/kaggle/working/chess-q8_0.gguf",
    n_ctx=4096,
    n_threads=8,
    temperature=0.35,
    top_p=0.9,
    repeat_penalty=1.15,
    verbose=False
)

def chat_gguf(prompt: str, max_tokens: int = 64) -> str:
    """
    Qwen-style GGUF chat, faithful to llama-cli usage
    """

    full_prompt = (
        "<|im_start|>user\n"
        ">> " + prompt.strip() + "\n"
        ">> <|im_end|>\n"
        ">> <|im_start|>assistant"
    )

    output = gguf_llm(
        full_prompt,
        max_tokens=max_tokens,
        stop=["<|im_end|>"],   # reverse-prompt equivalent
    )

    text = output["choices"][0]["text"].strip()

    # Safety cleanup: stop at first sentence end if needed
    for end in [".", "?", "!"]:
        if end in text:
            text = text[:text.rfind(end) + 1]
            break

    return text


llama_context: n_ctx_per_seq (4096) < n_ctx_train (40960) -- the full capacity of the model will not be utilized


## load the sample data with stockfish labels

In [24]:
import json

json_text = r"""
[
  {
    "before": {
      "fen": "r2qk2r/ppp2ppp/2n5/3n4/Q1B5/P1P1BP2/5P1P/R3K2R w KQkq - 0 14",
      "stockfish_analysis": "[White POV]: 86.87% cp:3.78 O-O-O Qf6 Rxd5 O-O Qc2 Ne7 Rd3 Ng6 Bb3 c5 Bxc5 Rfc8 Be3 Ne5 (Depth: 21)\n[White POV]: 85.19% cp:3.5 Rd1 Nxc3 Rxd8+ Rxd8 Qc2 Rd1+ Qxd1 Nxd1 Kxd1 Ke7 Bg5+ f6 Re1+ Kd7 Be3 Rd8 Kc1 g6 f4 f5 (Depth: 21)\n[White POV]: 31.53% cp:-1.55 Qb3 Nxe3 Bxf7+ Kf8 fxe3 Qe7 Bd5 Qxe3+ Kf1 Re8 Qc4 g6 Rd1 Re5 Qd3 Qc5 Bxc6 Qxc6 Qd8+ Kf7 (Depth: 21)",
      "eval": "86.87",
      "player_to_move": "White"
    },
    "after": {
      "fen": "r2qk2r/ppp2ppp/2n5/3B4/Q7/P1P1BP2/5P1P/R3K2R b KQkq - 0 14",
      "stockfish_analysis": "[White POV]: 22.88% cp:-2.43 Qxd5 Qe4+ Qxe4 fxe4 O-O-O Ke2 Rhe8 Rhg1 g6 Rg4 Ne5 Rf4 Rd7 Rd1 Re6 Rd5 b6 Bd4 f6 h4 (Depth: 19)\n[White POV]: 89.56% cp:4.3 O-O Bxc6 bxc6 O-O f5 Rfd1 Qe8 Qf4 Rf6 Kh1 Qh5 Rg1 h6 h4 Rg6 (Depth: 19)\n[White POV]: 92.58% cp:5.05 Qf6 Bxc6+ bxc6 Qe4+ Qe6 Qxe6+ fxe6 Rg1 g6 Ke2 Rf8 c4 O-O-O Rad1 Rxd1 Rxd1 Rf5 Rd3 Ra5 c5 (Depth: 19)",
      "eval": "22.88",
      "player_to_move": "Black"
    },
    "best_move": "O-O-O",
    "played_move": "Bxd5",
    "move_evaluation": "This was a blunder move. The position significantly worsened for White after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": true,
      "hanging_piece_details": [
        "bishop on d5"
      ],
      "king_safety_status": false,
      "center_control_status": true,
      "center_control_details": "Center control weakened",
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 1,
    "move_type": "blunder"
  },
  {
    "before": {
      "fen": "rnbq1rk1/p1p1bppp/1p2pn2/3p4/2PP1B2/2N1PN2/PP3PPP/R2QKB1R w KQ - 0 7",
      "stockfish_analysis": "[White POV]: 53.74% cp:0.3 cxd5 Nxd5 Nxd5 Qxd5 Bd3 Qa5+ Kf1 Ba6 Bxc7 Nd7 Bf4 Rac8 e4 Rfd8 Bxa6 Qxa6+ Qe2 Qxe2+ Kxe2 Rc2+ (Depth: 21)\n[White POV]: 52.99% cp:0.24 Bd3 dxc4 Bxc4 Bb7 O-O a6 Rc1 b5 Be2 Nbd7 Ne5 Nxe5 dxe5 Ne4 Nxe4 Bxe4 Qxd8 Rfxd8 Rxc7 Rd2 (Depth: 21)\n[White POV]: 52.74% cp:0.22 Rc1 Bb7 cxd5 Nxd5 Nxd5 Qxd5 a3 Rc8 Bc4 Qa5+ Nd2 c5 dxc5 bxc5 Qe2 Rd8 Bd3 c4 Rxc4 e5 (Depth: 21)",
      "eval": "53.74",
      "player_to_move": "White"
    },
    "after": {
      "fen": "rnbq1rk1/p1p1bppp/1p2pn2/3p4/2PP1B2/2NBPN2/PP3PPP/R2QK2R b KQ - 1 7",
      "stockfish_analysis": "[White POV]: 52.62% cp:0.21 dxc4 Bxc4 Bb7 Qe2 a6 Rd1 b5 Bb3 Qc8 O-O Nbd7 Rfe1 Re8 Rc1 c5 e4 cxd4 Nxd4 Nc5 e5 (Depth: 21)\n[White POV]: 53.36% cp:0.27 Ba6 cxd5 Nxd5 Nxd5 Qxd5 O-O Qb7 Rc1 c5 b4 Bxd3 Qxd3 Rc8 dxc5 bxc5 bxc5 Nd7 Rb1 Qc6 Nd4 (Depth: 21)\n[White POV]: 54.23% cp:0.34 Bb7 cxd5 Nxd5 Nxd5 Qxd5 O-O c5 Qb1 cxd4 Bxh7+ Kh8 Re1 Nd7 Bc2 Qh5 Be4 Bxe4 Qxe4 Nc5 Qxd4 (Depth: 21)",
      "eval": "52.62",
      "player_to_move": "Black"
    },
    "best_move": "cxd5",
    "played_move": "Bd3",
    "move_evaluation": "This was a normal move. The position slightly changed for White after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 1,
    "move_type": "normal"
  },
  {
    "before": {
      "fen": "rnb1kb1r/pp3ppp/1qp2p2/8/3P4/8/PPPB1PPP/R2QKBNR w KQkq - 0 8",
      "stockfish_analysis": "[White POV]: 57.68% cp:0.62 Nf3 Be6 Be2 Bb4 c3 Bd6 O-O Qc7 Rc1 O-O c4 Nd7 h3 Rfd8 b4 Qb8 Qc2 (Depth: 21)\n[White POV]: 53.74% cp:0.3 c3 Bf5 Bc4 Qxb2 Nf3 Qc2 Qe2+ Qe4 Qxe4+ Bxe4 O-O Kd7 Bxf7 Na6 Bh5 Bd6 Rfe1 Bd5 Bg4+ Kd8 (Depth: 21)\n[White POV]: 50.87% cp:0.07 Bc4 Qxd4 Qe2+ Qe5 Be3 Qa5+ c3 Be7 Bf4 Nd7 Nf3 Nb6 Bd6 Be6 Bxe6 Bxd6 Bb3+ Be5 O-O-O (Depth: 21)",
      "eval": "57.68",
      "player_to_move": "White"
    },
    "after": {
      "fen": "rnb1kb1r/pp3ppp/1qp2p2/8/3P4/1P6/P1PB1PPP/R2QKBNR b KQkq - 0 8",
      "stockfish_analysis": "[White POV]: 42.67% cp:-0.59 Qxd4 Nf3 Qd8 Bc4 Bd6 Qe2+ Qe7 a4 Qxe2+ Kxe2 O-O Be3 Re8 Rhd1 Bb4 (Depth: 21)\n[White POV]: 49.25% cp:-0.06 Bd6 Nf3 O-O c4 Ba3 Bc3 Bb4 Bxb4 Re8+ Be2 Qxb4+ Qd2 a5 Rd1 Nd7 O-O Qxd2 Rxd2 Nb6 d5 (Depth: 21)\n[White POV]: 50.99% cp:0.08 Be6 Nf3 c5 Be3 cxd4 Qxd4 Bb4+ c3 Qxd4 Bxd4 Ba3 Bb5+ Nc6 O-O O-O Bxc6 bxc6 Be3 a5 Nd4 (Depth: 21)",
      "eval": "42.67",
      "player_to_move": "Black"
    },
    "best_move": "Nf3",
    "played_move": "b3",
    "move_evaluation": "This was a mistake move. The position worsened for White after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": true,
      "hanging_piece_details": [
        "pawn on d4"
      ],
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 1,
    "move_type": "inaccuracy"
  },
  {
    "before": {
      "fen": "r4r1k/pp1nQppp/1qp2p2/5b2/2BP4/1PP5/P2B1PPP/R3K1NR w KQ - 3 13",
      "stockfish_analysis": "[White POV]: 91.13% cp:4.66 Ne2 Rfe8 Qd6 Qd8 O-O Nb6 Qxd8 Raxd8 Ng3 Nxc4 bxc4 Bd3 Rfd1 Bxc4 a4 b5 axb5 cxb5 h4 h6 (Depth: 21)\n[White POV]: 90.02% cp:4.4 Qd6 Qa5 Qg3 Qa3 Bc1 Rfe8+ Ne2 Qe7 Qf3 Bg6 Bd2 Be4 Qe3 Bxg2 Rg1 Qf8 Qg3 Be4 h4 b5 (Depth: 21)\n[White POV]: 89.84% cp:4.36 Qa3 Qc7 Nf3 Rae8+ Kf1 b5 Be2 c5 Re1 b4 Qc1 Rxe2 Kxe2 Kg8 Kf1 bxc3 Qxc3 cxd4 Qxc7 (Depth: 21)",
      "eval": "91.13",
      "player_to_move": "White"
    },
    "after": {
      "fen": "r4r1k/pp1nQppp/1qp2p2/5b2/2BP4/1PP5/P2B1PPP/2KR2NR b - - 4 13",
      "stockfish_analysis": "[White POV]: 80.99% cp:2.9 c5 Re1 cxd4 Nf3 dxc3 Bxc3 Rac8 Qb4 a5 Qxb6 Nxb6 Nd4 Bg6 Kb2 Nxc4+ bxc4 Rxc4 Nb5 h6 Rc1 (Depth: 21)\n[White POV]: 83.61% cp:3.26 Rfe8 Qa3 Qc7 Ne2 b5 Bf4 Qc8 Ng3 Nb6 Bd3 Bxd3 Rxd3 Qg4 Bd2 a5 Qb2 a4 b4 f5 Rf3 (Depth: 21)\n[White POV]: 84.48% cp:3.39 Qa5 Kb2 Rfe8 Qd6 b5 Bf1 c5 Nf3 c4 d5 Be4 Nd4 Ne5 Qb4 Qd8 Nxb5 Qxd5 (Depth: 21)",
      "eval": "80.99",
      "player_to_move": "Black"
    },
    "best_move": "Ne2",
    "played_move": "O-O-O",
    "move_evaluation": "This was a mistake move. The position worsened for White after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": false,
      "king_safety_status": true,
      "king_safety_details": "Pawn shield in front of the king was weakened",
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 1,
    "move_type": "mistake"
  },
  {
    "before": {
      "fen": "r4r1k/2qn1ppp/2p2p2/pp3b2/3P2P1/QPP2N1P/P2B1P2/2KR1B1R b - - 0 18",
      "stockfish_analysis": "[White POV]: 76.49% cp:2.36 Be4 Bg2 b4 Qb2 a4 Rhe1 axb3 axb3 f5 gxf5 bxc3 Bxc3 Bd5 Bh1 Ra6 Ne5 Rfa8 Bxd5 cxd5 Nxd7 (Depth: 21)\n[White POV]: 76.58% cp:2.37 b4 Qb2 Be4 Bg2 a4 Rhe1 f5 gxf5 bxc3 Bxc3 Bd5 Bh1 f6 Qc2 axb3 axb3 Rfb8 Nd2 Bxh1 Rxh1 (Depth: 21)\n[White POV]: 79.08% cp:2.66 Be6 Qb2 c5 dxc5 Nxc5 Nd4 b4 Kb1 a4 Nxe6 fxe6 cxb4 axb3 axb3 Ne4 Bc4 Nxf2 Rhe1 Nxd1 Rxd1 (Depth: 21)",
      "eval": "76.49",
      "player_to_move": "Black"
    },
    "after": {
      "fen": "r4r1k/2qn1ppp/2p2p2/pp6/3Pb1P1/QPP2N1P/P2B1P2/2KR1B1R w - - 1 19",
      "stockfish_analysis": "[White POV]: 75.95% cp:2.3 Bg2 b4 Qb2 a4 Rhe1 axb3 axb3 f5 gxf5 bxc3 Bxc3 Bd5 Bh1 f6 Re3 Rfb8 Nd2 Nb6 Bf3 Bxf3 (Depth: 18)\n[White POV]: 65.81% cp:1.31 Be2 c5 Rhe1 Qb7 d5 b4 Qb2 a4 Nh4 axb3 axb3 Bxd5 c4 Be4 f3 Ra3 fxe4 Rfa8 Kc2 Ra2 (Depth: 18)\n[White POV]: 42.92% cp:-0.57 Rg1 Bxf3 Re1 Qh2 Bd3 Rfe8 Be3 b4 Qb2 a4 cxb4 axb3 Qxb3 Bd5 (Depth: 18)",
      "eval": "75.95",
      "player_to_move": "White"
    },
    "best_move": "Be4",
    "played_move": "Be4",
    "move_evaluation": "This was a normal move. The position slightly changed for Black after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": true,
      "center_control_details": "Center control weakened",
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": true,
      "hanging_piece_details": [
        "knight on f3"
      ],
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 2,
    "move_type": "normal"
  },
  {
    "before": {
      "fen": "r4r1k/2qn1ppp/2p2p2/p7/1p1Pb1P1/QPP2N1P/P2B1PB1/2KR3R w - - 0 20",
      "stockfish_analysis": "[White POV]: 75.67% cp:2.27 Qb2 a4 Rhe1 f5 gxf5 axb3 axb3 bxc3 Bxc3 Bd5 Bh1 f6 Nd2 Bxh1 Rxh1 Nb6 Rhe1 Rfb8 Qc2 Nd5 (Depth: 19)\n[White POV]: 44.27% cp:-0.46 cxb4 axb4 Qxb4 Rxa2 Rde1 c5 Qc3 Bd5 Nh4 Bxg2 Nxg2 Qa7 d5 c4 Qxc4 Ra8 Ne3 Ne5 Qc3 Ra1+ (Depth: 19)\n[White POV]: 15.7% cp:-3.36 Rhe1 Bxf3 cxb4 Bxg2 bxa5 Bf3 Bb4 Bxd1 Bd6 Qb7 Bxf8 Nxf8 Re7 Qb5 Kxd1 Ne6 Qa4 Qf1+ Kc2 Qxf2+ (Depth: 19)",
      "eval": "75.67",
      "player_to_move": "White"
    },
    "after": {
      "fen": "r4r1k/2qn1ppp/2p2p2/p7/1P1Pb1P1/QP3N1P/P2B1PB1/2KR3R b - - 0 20",
      "stockfish_analysis": "[White POV]: 32.3% cp:-1.48 axb4 Qxb4 c5 Qc3 Rxa2 Rde1 f5 Rxe4 fxe4 Ng5 Qa7 d5 c4 Kd1 Qxf2 Bxe4 Nb6 Re1 cxb3 Re2 (Depth: 21)\n[White POV]: 82.2% cp:3.06 Rfb8 Rhe1 axb4 Qb2 f5 Rxe4 fxe4 Ng5 c5 Bxe4 Ra5 Nxf7+ Kg8 Ne5 Nxe5 dxe5 c4 bxc4 Ra3 Kb1 (Depth: 21)\n[White POV]: 82.34% cp:3.08 a4 bxa4 Rfb8 Ne1 c5 Qg3 Qb7 Bxe4 Qxe4 Nc2 Rxa4 Rhe1 Qa8 dxc5 Nxc5 bxc5 Rxa2 Bc3 (Depth: 21)",
      "eval": "32.3",
      "player_to_move": "Black"
    },
    "best_move": "Qb2",
    "played_move": "cxb4",
    "move_evaluation": "This was a blunder move. The position significantly worsened for White after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": true,
      "pawn_structure_details": {
        "doubled": [],
        "isolated": [
          "a-file",
          "c-file"
        ],
        "backward": []
      }
    },
    "position_features_black": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": true,
      "pawn_structure_details": {
        "doubled": [
          "b-file"
        ],
        "isolated": [
          "d-file"
        ],
        "backward": [
          "b-file"
        ]
      }
    },
    "move_number": 2,
    "move_type": "blunder"
  },
  {
    "before": {
      "fen": "7k/2q2ppp/5p2/2n5/1Q2R1PN/1P5P/2rB1P2/3K3R b - - 0 27",
      "stockfish_analysis": "[White POV]: 95.41% cp:6.07 Rxd2+ Kxd2 h6 Rd4 Qa7 Rd1 g6 Nf3 Kg7 Qc4 Ne6 b4 g5 Rd3 Qxf2+ Kc1 h5 gxh5 Nf4 (Depth: 20)\n[White POV]: 97.27% cp:7.15 g6 Re8+ Kg7 Kxc2 Qc6 Re3 Ne6+ Qc3 Qxh1 Nf3 Qf1 Kb2 Qa6 Rxe6 fxe6 Qc4 Qa7 Bc3 Qxf2+ Nd2 (Depth: 20)\n[White POV]: 97.63% cp:7.44 Rc1+ Bxc1 h5 Rc4 Qd8+ Rd4 Qa8 Qxc5 hxg4 Kc2 Kg8 Qc4 Qxh1 Rd8+ Kh7 Qd3+ g6 Qc4 Kg7 (Depth: 20)",
      "eval": "95.41",
      "player_to_move": "Black"
    },
    "after": {
      "fen": "7k/2q2ppp/5p2/8/1Q2n1PN/1P5P/2rB1P2/3K3R w - - 0 28",
      "stockfish_analysis": "Mate in 1 moves: Qf8# (Depth: 21)\n[White POV]: 96.33% cp:6.54 Qxe4 Rxd2+ Kxd2 h5 Nf3 Qb6 Qd4 Qc6 Rc1 Qa8 Qc3 Kh7 gxh5 g6 Nd4 Qg2 hxg6+ Kg7 Nf5+ Kxg6 (Depth: 21)\n[White POV]: 90.63% cp:4.54 Bc3 Nxc3+ Kxc2 Nb5+ Kb1 Qb8 Nf5 g6 Rc1 gxf5 gxf5 Kg7 (Depth: 21)",
      "eval": "100.0",
      "player_to_move": "White"
    },
    "best_move": "Rxd2+",
    "played_move": "Nxe4",
    "move_evaluation": "This is unavoidable checkmate on the board,The move is Blunder to hang the opponent in an unavoidable checkmate after this move.",
    "checkmate": {
      "unavoidable_checkmate": true,
      "mate_length": 1,
      "mated_side": "Black"
    },
    "position_features_white": {
      "hanging_piece": true,
      "hanging_piece_details": [
        "knight on e4"
      ],
      "king_safety_status": false,
      "center_control_status": true,
      "center_control_details": "Center control weakened",
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": true,
      "hanging_piece_details": [
        "pawn on f2"
      ],
      "king_safety_status": false,
      "center_control_status": true,
      "center_control_details": "Center control weakened",
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 3,
    "move_type": "blunder"
  },
  {
    "before": {
      "fen": "r1b1r1k1/1p1n1ppp/p3pq2/3p4/P1PP4/3BPN2/5PPP/1R1Q1RK1 b - - 0 14",
      "stockfish_analysis": "[White POV]: 54.85% cp:0.39 dxc4 Bxc4 b6 d5 e5 Ba2 Nc5 a5 b5 Rc1 Nb7 Qd2 Nd6 Rc6 Bd7 Rb6 Bg4 Rc1 Bxf3 gxf3 (Depth: 21)\n[White POV]: 66.48% cp:1.37 Qd8 Qc2 g6 c5 Qc7 Rfc1 e5 Bf1 exd4 Nxd4 Ne5 Qb3 Nc6 Qxd5 Rd8 Qc4 Nxd4 exd4 Bd7 g3 (Depth: 21)\n[White POV]: 68.24% cp:1.53 g6 cxd5 exd5 Qb3 Qd6 Rfc1 Rb8 h3 Re6 a5 Nf6 Ne5 Qd8 Qa3 Ne8 Rc5 Rd6 (Depth: 21)",
      "eval": "54.85",
      "player_to_move": "Black"
    },
    "after": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3pp3/P1PP4/3BPN2/5PPP/1R1Q1RK1 w - - 0 15",
      "stockfish_analysis": "[White POV]: 70.68% cp:1.76 dxe5 Nxe5 Nxe5 Rxe5 cxd5 Bf5 Qb3 Bxd3 Qxd3 Rd8 Rfd1 Rdxd5 Qxd5 Rxd5 Rxd5 h6 a5 Qc3 h3 Qc7 (Depth: 21)\n[White POV]: 69.84% cp:1.68 Nxe5 Nxe5 dxe5 Rxe5 cxd5 Bf5 Qb3 Bxd3 Qxd3 Rd8 Rfd1 Rdxd5 Qxd5 Rxd5 Rxd5 h6 a5 Qe7 Rdd1 Kh7 (Depth: 21)\n[White POV]: 56.83% cp:0.55 Be2 dxc4 Bxc4 exd4 Nxd4 Ne5 Bd5 Rb8 h3 b5 axb5 axb5 Nxb5 Bf5 e4 Bd7 Bxf7+ Qxf7 Nd6 Rxb1 (Depth: 21)",
      "eval": "70.68",
      "player_to_move": "White"
    },
    "best_move": "dxc4",
    "played_move": "e5",
    "move_evaluation": "This was a mistake move. The position worsened for Black after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": true,
      "hanging_piece_details": [
        "pawn on d5"
      ],
      "king_safety_status": false,
      "center_control_status": true,
      "center_control_details": "Center control weakened",
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 2,
    "move_type": "mistake"
  },
  {
    "before": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3P4/P2Pp3/3BPN2/5PPP/1R1Q1RK1 w - - 0 16",
      "stockfish_analysis": "[White POV]: 33.29% cp:-1.39 Be2 exf3 Bxf3 b5 e4 Qb6 Qd2 Nf8 h4 Bd7 h5 Rac8 a5 Qf6 e5 Qf5 Rfe1 h6 Rbc1 Rc4 (Depth: 21)\n[White POV]: 31.64% cp:-1.54 Bxe4 Rxe4 Qc2 Re8 e4 b5 h3 Rb8 axb5 axb5 Rb3 Qa6 Rc1 h6 Rcb1 Qg6 Re3 b4 (Depth: 21)\n[White POV]: 27.19% cp:-1.97 Bc4 exf3 Qxf3 Qg6 Qg3 Qf5 Qc7 Nf6 d6 Be6 Rxb7 Red8 Bxe6 Qxe6 Qe7 Rxd6 Qxe6 fxe6 Rc1 Rd7 (Depth: 21)",
      "eval": "33.29",
      "player_to_move": "White"
    },
    "after": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3P4/P2PB3/4PN2/5PPP/1R1Q1RK1 b - - 0 16",
      "stockfish_analysis": "[White POV]: 31.64% cp:-1.54 Rxe4 Qc2 Re8 e4 b5 axb5 axb5 Rfe1 Rb8 Qc6 b4 Ne5 Qe7 Nd3 Nf6 d6 Qd8 e5 Bb7 Qc7 (Depth: 20)\n[White POV]: 83.47% cp:3.24 Qd8 Bd3 Nf6 Qc2 Qxd5 Bc4 Qh5 Ne5 Re7 Rb6 Bf5 Qb2 Ng4 Nxg4 Bxg4 Rxb7 (Depth: 20)\n[White POV]: 85.13% cp:3.49 Qd6 Nd2 b5 Bf3 Nb6 axb5 axb5 Qb3 Bd7 Ne4 Qxd5 Qxd5 Nxd5 Nc5 Bc6 (Depth: 20)",
      "eval": "31.64",
      "player_to_move": "Black"
    },
    "best_move": "Be2",
    "played_move": "Bxe4",
    "move_evaluation": "This was a normal move. The position slightly changed for White after this move.",
    "checkmate": {
      "unavoidable_checkmate": false
    },
    "position_features_white": {
      "hanging_piece": false,
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "position_features_black": {
      "hanging_piece": true,
      "hanging_piece_details": [
        "bishop on e4"
      ],
      "king_safety_status": false,
      "center_control_status": false,
      "development_status": false,
      "pawn_structure_status": false
    },
    "move_number": 3,
    "move_type": "normal"
  }
]
"""

samples = json.loads(json_text)

## testing the model with huggingface format **on gpu**

### simple method

In [25]:
for sample in samples:
    simple_method(sample,chat_qwen3_direct)

### chained method

In [26]:
for sample in samples:
    full_llm_pipeline(sample,chat_qwen3_direct)

## testing the model with gguf format **on cpu**

### simple method

In [27]:
# print(simple_method(samples[6],chat_gguf))

### chained method

In [28]:
# import json

# result = full_llm_pipeline(samples[6], chat_gguf)
# print(json.dumps(result, indent=2, ensure_ascii=False))